# Long Short-term Memory for Sentiment Classification

This notebook uses LSTM neural network on the [IMDB sentiment classification](https://keras.io/api/datasets/imdb/) task. This is a dataset for binary sentiment classification. 25,000 highly polar movie reviews are provided for training.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence


## Dataset

### Get the data
IMDB sentiment dataset is available in keras.datasets.

In [2]:
help(imdb.load_data)

Help on function load_data in module tensorflow.python.keras.datasets.imdb:

load_data(path='imdb.npz', num_words=None, skip_top=0, maxlen=None, seed=113, start_char=1, oov_char=2, index_from=3, **kwargs)
    Loads the [IMDB dataset](https://ai.stanford.edu/~amaas/data/sentiment/).
    
    This is a dataset of 25,000 movies reviews from IMDB, labeled by sentiment
    (positive/negative). Reviews have been preprocessed, and each review is
    encoded as a list of word indexes (integers).
    For convenience, words are indexed by overall frequency in the dataset,
    so that for instance the integer "3" encodes the 3rd most frequent word in
    the data. This allows for quick filtering operations such as:
    "only consider the top 10,000 most
    common words, but eliminate the top 20 most common words".
    
    As a convention, "0" does not stand for a specific word, but instead is used
    to encode any unknown word.
    
    Args:
      path: where to cache the data (relative to `~

In [3]:
max_features = 22000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

Loading data...
17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


25000 train sequences
25000 test sequences


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


### Data Preprocessing

Keras has already preprocessed the data

In [4]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
X_train[1]

Pad sequences (samples x time)
X_train shape: (25000, 80)
X_test shape: (25000, 80)


array([ 125,   68,    2, 6853,   15,  349,  165, 4362,   98,    5,    4,
        228,    9,   43,    2, 1157,   15,  299,  120,    5,  120,  174,
         11,  220,  175,  136,   50,    9, 4373,  228, 8255,    5,    2,
        656,  245, 2350,    5,    4, 9837,  131,  152,  491,   18,    2,
         32, 7464, 1212,   14,    9,    6,  371,   78,   22,  625,   64,
       1382,    9,    8,  168,  145,   23,    4, 1690,   15,   16,    4,
       1355,    5,   28,    6,   52,  154,  462,   33,   89,   78,  285,
         16,  145,   95], dtype=int32)

## RNN

### Build the RNN model

In [5]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 32))
model.add(layers.SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

### Inspect the model

Use the `.summary` method to print a simple description of the model

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          704000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 64)                6208      
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 710,273
Trainable params: 710,273
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [9]:
EPOCHS = 16
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/16
313/313 [==============================] - 13s 43ms/step - loss: 0.2895 - accuracy: 0.8867 - val_loss: 0.6031 - val_accuracy: 0.8064
Epoch 2/16
313/313 [==============================] - 13s 42ms/step - loss: 0.2803 - accuracy: 0.8917 - val_loss: 0.4799 - val_accuracy: 0.8188
Epoch 3/16
313/313 [==============================] - 13s 42ms/step - loss: 0.2629 - accuracy: 0.8991 - val_loss: 0.5991 - val_accuracy: 0.8022
Epoch 4/16
313/313 [==============================] - 13s 43ms/step - loss: 0.2563 - accuracy: 0.9010 - val_loss: 0.4988 - val_accuracy: 0.8156


In [10]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 81.93%


## RNN using the entire sequence instead of the last output

In [11]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16, input_length=maxlen))
model.add(layers.SimpleRNN(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 80, 16)            352000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 80, 64)            5184      
_________________________________________________________________
flatten (Flatten)            (None, 5120)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5121      
Total params: 362,305
Trainable params: 362,305
Non-trainable params: 0
_________________________________________________________________


In [12]:
EPOCHS = 16
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/16
313/313 [==============================] - 14s 40ms/step - loss: 0.7153 - accuracy: 0.5066 - val_loss: 0.6879 - val_accuracy: 0.5378
Epoch 2/16
313/313 [==============================] - 12s 40ms/step - loss: 0.6036 - accuracy: 0.6481 - val_loss: 0.4672 - val_accuracy: 0.7942
Epoch 3/16
313/313 [==============================] - 12s 39ms/step - loss: 0.3680 - accuracy: 0.8361 - val_loss: 0.3703 - val_accuracy: 0.8328
Epoch 4/16
313/313 [==============================] - 12s 40ms/step - loss: 0.2881 - accuracy: 0.8780 - val_loss: 0.3456 - val_accuracy: 0.8458
Epoch 5/16
313/313 [==============================] - 12s 40ms/step - loss: 0.2498 - accuracy: 0.8993 - val_loss: 0.3635 - val_accuracy: 0.8444
Epoch 6/16
313/313 [==============================] - 12s 40ms/step - loss: 0.2114 - accuracy: 0.9176 - val_loss: 0.3729 - val_accuracy: 0.8464


In [13]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 83.88%


## LSTM

### Build the model

In [18]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16))
model.add(layers.LSTM(264, dropout=0.25, recurrent_dropout=0.25))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

### Inspect the model

Use the `.summary` method to print a simple description of the model

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 16)          320000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               74240     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 394,369
Trainable params: 394,369
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [20]:
EPOCHS = 16
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/16
313/313 [==============================] - 246s 785ms/step - loss: 0.2302 - accuracy: 0.9111 - val_loss: 0.4248 - val_accuracy: 0.8350
Epoch 2/16
313/313 [==============================] - 247s 788ms/step - loss: 0.2136 - accuracy: 0.9201 - val_loss: 0.3660 - val_accuracy: 0.8514
Epoch 3/16
313/313 [==============================] - 247s 789ms/step - loss: 0.2004 - accuracy: 0.9244 - val_loss: 0.4205 - val_accuracy: 0.8464
Epoch 4/16
313/313 [==============================] - 246s 786ms/step - loss: 0.1885 - accuracy: 0.9284 - val_loss: 0.4122 - val_accuracy: 0.8446


In [ ]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 83.91%


## Stacked LSTM

### Build the model

In [ ]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16))
model.add(layers.LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 16)          320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 128)         74240     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 525,953
Trainable params: 525,953
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [ ]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/32
313/313 [==============================] - 45s 132ms/step - loss: 0.5047 - accuracy: 0.7484 - val_loss: 0.4167 - val_accuracy: 0.8234
Epoch 2/32
313/313 [==============================] - 44s 139ms/step - loss: 0.3336 - accuracy: 0.8612 - val_loss: 0.5216 - val_accuracy: 0.8244
Epoch 3/32
313/313 [==============================] - 43s 138ms/step - loss: 0.2802 - accuracy: 0.8855 - val_loss: 0.3630 - val_accuracy: 0.8412
Epoch 4/32
313/313 [==============================] - 45s 144ms/step - loss: 0.2499 - accuracy: 0.9029 - val_loss: 0.5825 - val_accuracy: 0.8138
Epoch 5/32
313/313 [==============================] - 43s 138ms/step - loss: 0.2296 - accuracy: 0.9099 - val_loss: 0.3626 - val_accuracy: 0.8428
Epoch 6/32
313/313 [==============================] - 46s 147ms/step - loss: 0.2098 - accuracy: 0.9198 - val_loss: 0.5436 - val_accuracy: 0.8294
Epoch 7/32
313/313 [==============================] - 45s 144ms/step - loss: 0.1998 - accuracy: 0.9255 - val_loss: 0.3799 - val_ac

In [ ]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 83.40%


## Bidirectional LSTM

In [ ]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16))
model.add(layers.􏲇􏱢􏱒􏱢􏱡􏱖􏱓􏱚􏱢􏱎􏱆􏱑􏱍􏲇􏱢􏱒􏱢􏱡􏱖􏱓􏱚􏱢􏱎􏱆􏱑􏱍Bidirectional(layers.LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

SyntaxError: invalid non-printable character U+10FC87 (<ipython-input-19-e958e204a9bd>, line 4)

### Train the model

In [ ]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))